<a href="https://colab.research.google.com/github/kmk028/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/blob/master/module1-statistics-probability-and-inference/LS_DS_131_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
### YOUR CODE STARTS HERE
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel
import pandas as pd
import numpy as np

In [87]:
missing_val = ['?']
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data',header=None, sep = ',',na_values = '?')

df.sample(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
56,republican,n,y,n,y,y,y,n,n,n,y,y,y,y,y,n,y
75,democrat,n,y,y,y,y,y,n,n,n,y,y,n,y,y,n,n
326,democrat,y,y,n,y,n,n,y,y,y,n,y,n,n,y,n,y
431,democrat,n,n,y,n,n,n,y,y,y,y,n,n,n,n,n,y
149,democrat,n,n,y,n,n,n,y,y,y,y,n,n,y,n,y,y
297,democrat,y,y,y,n,n,y,y,y,y,y,n,n,NaN,n,y,NaN
322,democrat,y,y,y,n,NaN,y,n,NaN,n,n,y,n,y,y,n,NaN
369,republican,n,y,n,y,y,y,n,n,n,n,n,n,y,y,n,y
285,democrat,n,n,y,n,n,y,y,y,y,y,y,n,n,n,y,y
411,democrat,y,n,y,n,n,y,y,y,y,y,y,n,n,n,n,y


In [88]:
df.isnull().sum()


0       0
1      12
2      48
3      11
4      11
5      15
6      11
7      14
8      15
9      22
10      7
11     21
12     31
13     25
14     17
15     28
16    104
dtype: int64

In [0]:
df.columns = ['Party','handicapped-infants','water-project-cost-sharing','adoption-of-the-budget-resolution','physician-fee-freeze','el-salvador-aid'
             ,'religious-groups-in-schools','anti-satellite-test-ban','aid-to-nicaraguan-contras','mx-missile','immigration','synfuels-corporation-cutback'
             ,'education-spending','superfund-right-to-sue','crime','duty-free-exports','export-administration-act-south-africa']

In [91]:
df.sample(10)

,Party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
310,republican,n,n,n,y,y,y,n,n,n,y,n,y,y,y,y,n
387,democrat,y,y,y,n,y,y,n,y,y,n,y,n,n,y,n,NaN
301,democrat,n,n,NaN,n,n,y,y,y,y,n,n,n,n,n,y,y
318,democrat,n,n,y,n,n,n,y,y,y,n,n,n,n,n,y,NaN
323,democrat,y,y,y,n,y,y,n,y,NaN,y,n,n,y,y,n,NaN
118,democrat,n,y,y,n,n,n,n,y,y,y,y,n,n,n,y,y
144,democrat,n,NaN,n,n,n,y,y,y,y,y,n,n,n,y,n,NaN
366,democrat,y,n,y,n,n,n,y,y,n,y,y,n,n,n,n,NaN
283,republican,n,n,n,y,y,y,n,n,n,n,n,y,y,y,n,y
117,republican,y,y,y,y,y,n,y,n,n,n,n,y,y,y,n,y


In [0]:
issue_df = df.drop(['Party'], axis=1)

In [99]:
for column in issue_df.columns:
  subdf = df[['Party',column]]
  subdf.dropna()
  #subdf.replace(['n', 'y'],[0,1])
  group1=(subdf[subdf['Party'] == 'democrat'][column])
  group2 = (subdf[subdf['Party'] == 'republican'][column])
  statistic, pvalue = ttest_ind(group1, group2)
  f"Issue, {column}. P-value {pvalue}."
  

TypeError: ignored